In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [21]:
# Step 1: Load annotations from the CSV file and sort it to match the image folder
csv_file = "/kaggle/input/plate-car-detection/Car_detection/train_labels.csv"  # Update the path to the CSV file
df = pd.read_csv(csv_file)
# annotations = annotations.sort_values(by='filename')



In [24]:
df.head(5)


,filename,width,height,class,xmin,ymin,xmax,ymax
0,Data-Images/Cars/105.jpg,684,293,license_plate,12,106,696,399
1,Data-Images/Cars/38.jpg,118,39,license_plate,794,485,912,524
2,Data-Images/Cars/136.jpg,173,74,license_plate,155,200,328,274
3,Data-Images/Cars/127.jpg,416,154,license_plate,191,54,607,208
4,Data-Images/Cars/21.jpg,238,63,license_plate,272,2,510,65


In [29]:
import pandas as pd


df['xmin']/=df['width']
df['ymin']/=df['height']
df['xmax']/=df['width']
df['ymax']/=df['height']
print(df)



                     filename  width  height          class      xmin  \
18     Data-Images/Cars/0.jpg    119      47  license_plate  0.041028   
148    Data-Images/Cars/1.jpg    209      68  license_plate  0.001809   
70    Data-Images/Cars/10.jpg    222      81  license_plate  0.004038   
201  Data-Images/Cars/100.jpg    199     109  license_plate  0.000707   
142  Data-Images/Cars/101.jpg    206      98  license_plate  0.000471   
..                        ...    ...     ...            ...       ...   
47    Data-Images/Cars/95.jpg    369     115  license_plate  0.002211   
15    Data-Images/Cars/96.jpg    281      75  license_plate  0.003989   
159   Data-Images/Cars/97.jpg     98      53  license_plate  0.012703   
122   Data-Images/Cars/98.jpg    188      84  license_plate  0.000707   
202   Data-Images/Cars/99.jpg    157      74  license_plate  0.004098   

         ymin      xmax      ymax  
18   0.123585  0.049432  0.144862  
148  0.082396  0.006593  0.097102  
70   0.021033  

In [34]:
# file = "/kaggle/input/data-images/train_labels.csv"
# df = pd.read_csv(file)
df["image_name"] = df["filename"].apply(lambda link: os.path.basename(link))
df = df.sort_values(by=["filename"])
df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_name
18,Data-Images/Cars/0.jpg,119,47,license_plate,0.041028,0.123585,0.049432,0.144862,0.jpg
148,Data-Images/Cars/1.jpg,209,68,license_plate,0.001809,0.082396,0.006593,0.097102,1.jpg
70,Data-Images/Cars/10.jpg,222,81,license_plate,0.004038,0.021033,0.008542,0.033379,10.jpg
201,Data-Images/Cars/100.jpg,199,109,license_plate,0.000707,0.004377,0.005732,0.013551,100.jpg
142,Data-Images/Cars/101.jpg,206,98,license_plate,0.000471,0.005727,0.005326,0.015931,101.jpg
...,...,...,...,...,...,...,...,...,...
47,Data-Images/Cars/95.jpg,369,115,license_plate,0.002211,0.031682,0.004921,0.040378,95.jpg
15,Data-Images/Cars/96.jpg,281,75,license_plate,0.003989,0.034133,0.007548,0.047467,96.jpg
159,Data-Images/Cars/97.jpg,98,53,license_plate,0.012703,0.041296,0.022907,0.060164,97.jpg
122,Data-Images/Cars/98.jpg,188,84,license_plate,0.000707,0.008929,0.006026,0.020833,98.jpg


In [30]:
# Step 2: Load and preprocess the images and bounding boxes
def load_images_and_annotations(annotations, images_directory):
    images = []
    bounding_boxes = []

    for index, row in annotations.iterrows():
        image_filename = os.path.basename(row['filename'])
        image_path = os.path.join(images_directory, image_filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)

        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']

        image_height, image_width = image.shape[:2]

        # Normalize bounding box values
        bounding_box = [xmin / image_width, ymin / image_height, (xmax - xmin) / image_width, (ymax - ymin) / image_height]
        bounding_boxes.append(bounding_box)

    return np.array(images), np.array(bounding_boxes)



In [31]:
images_directory = "/kaggle/input/plate-car-detection/Car_detection/Cars"  # Update the path to the image folder
images, bounding_boxes = load_images_and_annotations(annotations, images_directory)




/tmp/ipykernel_28/1028290089.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images), np.array(bounding_boxes)


In [37]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    df,
    directory="/kaggle/input/plate-car-detection/Car_detection/Cars",
    x_col="image_name",
    y_col=["xmin", "ymin", "xmax", "ymax"],
    target_size=(229, 229),
    batch_size=16,
    class_mode='raw',
    subset="training")

validation_generator = datagen.flow_from_dataframe(
    df,
    directory="/kaggle/input/plate-car-detection/Car_detection/Cars",
    x_col="image_name",
    y_col=["xmin", "ymin", "xmax", "ymax"],
    target_size=(229, 229),
    batch_size=16,
    class_mode='raw',
    subset="validation")

Found 214 validated image filenames.
Found 23 validated image filenames.


In [40]:

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(299,299,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dropout(0.25)(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dropout(0.25)(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 4s 0us/step


In [ ]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))


In [ ]:
tfb = TensorBoard('object_detection')
history = model.fit(train_generator,batch_size=16,epochs=50,
                    validation_data=validation_generator,callbacks=[tfb])

In [ ]:
model.evaluate_generator(validation_generator)

In [ ]:
images_folder = "/kaggle/input/testss/car_with_diff"
image_files = os.listdir(images_folder)

for image_file in image_files:
    img_path = os.path.join(images_folder, image_file)
    img = cv2.resize(cv2.imread(img_path) / 255.0, dsize=(WIDTH, HEIGHT))
    y_hat = model.predict(img.reshape(1, WIDTH, HEIGHT, 3)).reshape(-1) * WIDTH

    xt, yt = int(y_hat[0]), int(y_hat[1])
    xb, yb = int(y_hat[2]), int(y_hat[3])

    img = cv2.cvtColor(img.astype(np.float32), cv2.COLOR_BGR2RGB)
    image = cv2.rectangle(img, (xt, yt), (xb, yb), (0, 0, 255), 2)
    plt.imshow(image)
    plt.show()

In [ ]:
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 15 Cropped image')
fig.show()

In [ ]:
#ocr
import pytesseract as pt
roi_uint8 = (roi * 255).astype(np.uint8)
text = pt.image_to_string(roi_uint8)
print(text)

Lets view some predictions and detected objects 